In [ ]:
# =============================================================================
# Federated Learning Comparison: Neural Network, SVM, and Logistic Regression
# =============================================================================
# This notebook compares three machine learning approaches using federated learning
# for disease classification based on spectral data
# =============================================================================

# Federated Learning Comparison Study

This notebook implements and compares three different machine learning approaches using federated learning:

1. **Neural Network (NN)** - Deep learning approach with fully connected layers
2. **Support Vector Machine (SVM)** - Linear classifier with hinge loss
3. **Logistic Regression (LR)** - Linear classifier with logistic loss

All models are trained using various federated learning strategies including FedAvg, FedProx, and FedAdam.

## Dataset
The dataset contains spectral data for disease classification with:
- Features: Spectral measurements (300 dimensions)
- Labels: Binary classification (diseased vs healthy)
- Partitioned across 5 federated clients

## Setup and Dependencies

In [ ]:
# Install required packages
!pip install -q flwr[simulation]

# Import all necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import flwr as fl
from flwr.client import NumPyClient, Client, ClientApp
from flwr.server import ServerApp, ServerAppComponents, ServerConfig
from flwr.simulation import run_simulation
from flwr.server.strategy import FedAvg, FedProx, FedAdam, FedOpt
from flwr.common import Context, Parameters, ndarrays_to_parameters, parameters_to_ndarrays
from scipy.interpolate import BSpline, make_lsq_spline
import warnings
warnings.filterwarnings('ignore')

print("All libraries imported successfully!")

## Data Loading and Preprocessing

Upload your CSV files containing the spectral data. The files should be named with 'diseased' or 'healthy' to automatically assign labels.

In [ ]:
# Upload data files
from google.colab import files
uploaded = files.upload()

# Process uploaded files and combine data
print("Processing uploaded files...")
all_data = []

for file in uploaded:
    # Read and transpose the data (assuming first row contains column names)
    df = pd.read_csv(file, header=None)
    df = df.T
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])
    df = df.apply(pd.to_numeric, errors='coerce')

    print(f"{file}: shape = {df.shape}")

    # Assign labels based on filename
    label = 1 if "diseased" in file.lower() else 0
    df["label"] = label
    all_data.append(df)

# Combine all datasets
df_all = pd.concat(all_data, ignore_index=True)
print(f"\nCombined dataset shape: {df_all.shape}")

# Extract features and labels
feature_cols = df_all.columns[2:-1]  # Remove ID columns and label
X = df_all[feature_cols].values  # Features (N, 300)
y = df_all['label'].values       # Labels (N,)

print(f"Features shape: {X.shape}")
print(f"Labels shape: {y.shape}")
print(f"Class distribution: {np.bincount(y)}")

## Configuration and Utility Functions

This section defines the configuration parameters and utility functions that will be used across all three federated learning approaches.

In [ ]:
# =============================================================================
# CONFIGURATION PARAMETERS
# =============================================================================

# Federated Learning Configuration
NUM_PARTITIONS = 5      # Number of federated clients
NUM_FOLDS = 20          # Number of cross-validation folds
NUM_ROUNDS = 10        # Number of federated learning rounds

# Model-specific parameters
NN_EPOCHS = 5          # Local epochs for Neural Network
NN_BATCH_SIZE = 32     # Batch size for Neural Network
NN_LEARNING_RATE = 0.001

# Model-specific strategy configurations
model_strategies = {
    'nn': {
        "FedAvg": {"no_values": [0]},
        "FedProx": {"mu_values": [0.01, 0.1, 1.0]},
        "FedAdam": {"eta_values": [0.005, 0.01, 0.05, 0.1]},
        "FedOpt": {"eta_values": [0.005, 0.01, 0.05, 0.1]},
    },
    'svm': {
        "FedAvg": {"no_values": [0]},
        "FedProx": {"mu_values": [0.01, 0.1, 1.0]},
        "FedAdam": {"eta_values": [0.1, 0.5, 2.0, 5.0]},
        "FedOpt": {"eta_values": [0.1, 0.5, 2.0, 5.0]},
    },
    'lr': {
        "FedAvg": {"no_values": [0]},
    "FedProx": {"mu_values": [0.01, 0.1, 1.0]},
    "FedAdam": {"eta_values": [1.0, 2.0, 3.0, 5.0]},
    "FedOpt": {"eta_values": [0.1, 0.2, 0.5, 1.0]},
    }
}

def get_strategies_for_model(model_type):
    """Get strategies configuration for a specific model type"""
    if model_type in model_strategies:
        return model_strategies[model_type]
    else:
        # Fallback to default configuration
        return {
            "FedAvg": {"no_values": [0]},
            "FedProx": {"mu_values": [0.1, 1.0]},
            "FedAdam": {"eta_values": [0.01, 0.1]},
            "FedOpt": {"eta_values": [0.01, 0.1]},
        }

print(f"Configuration loaded:")
print(f"- Clients: {NUM_PARTITIONS}")
print(f"- Folds: {NUM_FOLDS}")
print(f"- FL Rounds: {NUM_ROUNDS}")
for model_type, strategies in model_strategies.items():
    print(f"\n{model_type.upper()} Model:")
    for strategy_name, params in strategies.items():
        param_type = list(params.keys())[0]
        param_values = list(params.values())[0]
        print(f"  {strategy_name}: {param_type} = {param_values}")

In [ ]:
# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================

def evaluate_model_sklearn(model, X, y):
    """Evaluate sklearn model and return comprehensive metrics"""
    try:
        # Get predictions
        y_pred = model.predict(X)

        # Calculate metrics
        accuracy = accuracy_score(y, y_pred)
        precision = precision_score(y, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y, y_pred, average='weighted', zero_division=0)

        return {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        }
    except Exception as e:
        print(f"Error in evaluation: {e}")
        return {'accuracy': 0, 'precision': 0, 'recall': 0, 'f1': 0}

def train_model_sklearn(model, X, y):
    """Train sklearn model with error handling"""
    try:
        model.fit(X, y)
        return True
    except Exception as e:
        print(f"Training error: {e}")
        return False

def get_parameters_sklearn(model):
    """Extract parameters from sklearn model"""
    if hasattr(model, 'coef_') and hasattr(model, 'intercept_'):
        return [model.coef_.flatten(), model.intercept_.flatten()]
    else:
        # For untrained models, return zeros
        return [np.zeros(300), np.zeros(1)]

def set_parameters_sklearn(model, parameters):
    """Set parameters for sklearn model"""
    if len(parameters) == 2:
        model.coef_ = parameters[0].reshape(1, -1)
        model.intercept_ = parameters[1]
        model.classes_ = np.array([0, 1])  # Binary classification

def compute_knot_vector(num_basis, spline_order):
    num_total_knots = num_basis + spline_order + 1
    num_internal_knots = num_total_knots - 2 * (spline_order + 1)
    internal_knots = np.linspace(0, 1, num_internal_knots + 2)[1:-1]
    t = np.concatenate((
        np.repeat(0.0, spline_order + 1),
        internal_knots,
        np.repeat(1.0, spline_order + 1)
    ))
    return t

def create_spline_features(X, num_basis=10, spline_order=3):
    N, D = X.shape
    x_domain = np.linspace(0, 1, D)
    t = compute_knot_vector(num_basis, spline_order)
    coeffs = np.zeros((N, num_basis))
    for i in range(N):
        y = X[i]
        spline = make_lsq_spline(x_domain, y, t, spline_order)
        coeffs[i] = spline.c
    return coeffs

def configure_model_hyperparameters(model_type, custom_config=None):
    """
    Configure hyperparameters for a specific model type

    Args:
        model_type: 'nn', 'svm', or 'lr'
        custom_config: Optional dictionary to override default values

    Returns:
        Dictionary of strategies with model-specific hyperparameters
    """

    # Get default configuration for the model
    default_config = get_strategies_for_model(model_type)

    # If custom configuration provided, merge it
    if custom_config:
        for strategy_name, params in custom_config.items():
            if strategy_name in default_config:
                default_config[strategy_name].update(params)
            else:
                default_config[strategy_name] = params

    return default_config

print("Utility functions defined successfully!")

## Data Partitioning for Federated Learning

The data is partitioned into federated clients with preprocessing steps including:
- Train/test split with stratification
- Standardization using StandardScaler
- B-spline feature engineering
- Distribution across federated clients

In [ ]:
# =============================================================================
# DATA PARTITIONING FUNCTION
# =============================================================================

def create_federated_data_partitions(X, y, fold_idx=0, apply_splines=True):
    """
    Create federated data partitions for a specific fold

    Args:
        X: Feature matrix
        y: Labels
        fold_idx: Current fold index for reproducible splits
        apply_splines: Whether to apply B-spline preprocessing

    Returns:
        Dictionary containing partitioned data for each client
    """

    print(f"\nCreating federated partitions for fold {fold_idx + 1}...")

    # Split data into train and test
    X_train_raw, X_test_raw, y_train, y_test = train_test_split(
        X, y, test_size=0.2, stratify=y, shuffle=True, random_state=fold_idx
    )

    print(f"Train size: {X_train_raw.shape[0]}, Test size: {X_test_raw.shape[0]}")

    # Partition training data across clients
    X_train_partitions_raw = np.array_split(X_train_raw, NUM_PARTITIONS)
    y_train_partitions = np.array_split(y_train, NUM_PARTITIONS)
    X_test_partitions_raw = np.array_split(X_test_raw, NUM_PARTITIONS)
    y_test_partitions = np.array_split(y_test, NUM_PARTITIONS)

    # Initialize data containers
    client_train_data = []
    client_val_data = []

    # Process each client's data
    for cid in range(NUM_PARTITIONS):
        print(f"Processing client {cid + 1}/{NUM_PARTITIONS}...")

        # Get raw data for this client
        X_train_client_raw = X_train_partitions_raw[cid]
        y_train_client = y_train_partitions[cid]
        X_test_client_raw = X_test_partitions_raw[cid]
        y_test_client = y_test_partitions[cid]

        # Apply B-spline preprocessing if requested
        if apply_splines:
            try:
                X_train_client_processed = create_spline_features(X_train_client_raw)
                X_test_client_processed = create_spline_features(X_test_client_raw)
            except:
                print(f"Spline processing failed for client {cid}, using raw features")
                X_train_client_processed = X_train_client_raw
                X_test_client_processed = X_test_client_raw
        else:
            X_train_client_processed = X_train_client_raw
            X_test_client_processed = X_test_client_raw

        # Standardize features
        scaler = StandardScaler()
        X_train_client_scaled = scaler.fit_transform(X_train_client_processed)
        X_test_client_scaled = scaler.transform(X_test_client_processed)

        # Store processed data
        client_train_data.append((X_train_client_scaled, y_train_client))
        client_val_data.append((X_test_client_scaled, y_test_client))

        print(f"Client {cid + 1} - Train: {X_train_client_scaled.shape}, Test: {X_test_client_scaled.shape}")

    # Create centralized data for comparison
    scaler_central = StandardScaler()
    if apply_splines:
        X_train_central = create_spline_features(X_train_raw)
        X_test_central = create_spline_features(X_test_raw)
    else:
        X_train_central = X_train_raw
        X_test_central = X_test_raw

    X_train_scaled_central = scaler_central.fit_transform(X_train_central)
    X_test_scaled_central = scaler_central.transform(X_test_central)

    return {
        'client_train_data': client_train_data,
        'client_val_data': client_val_data,
        'centralized_train': (X_train_scaled_central, y_train),
        'centralized_test': (X_test_scaled_central, y_test),
        'input_dim': X_train_client_scaled.shape[1]
    }

print("Data partitioning function ready!")

## Neural Network Implementation

This section implements a simple feedforward neural network for binary classification with PyTorch, including:
- Network architecture definition
- Training and evaluation functions
- Federated learning client implementation
- Parameter serialization for FL communication

In [ ]:
# =============================================================================
# NEURAL NETWORK MODEL DEFINITION
# =============================================================================

def get_device():
    """Get available device, with fallback to CPU"""
    try:
        if torch.cuda.is_available():
            return torch.device("cuda")
        else:
            return torch.device("cpu")
    except:
        return torch.device("cpu")

# Initialize device
device = get_device()
print("Using device:", device)

# Only print CUDA info if CUDA is actually available
if device.type == "cuda":
    try:
        print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
        print(f"CUDA Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.1f} GB")
    except:
        print("CUDA device info not available")

print("Device handling setup complete!")

class SimpleNN(nn.Module):
    """
    Simple feedforward neural network for binary classification
    Architecture: Input -> Hidden(64) -> BatchNorm -> Dropout -> Hidden(32) -> Output(2)
    """

    def __init__(self, input_size):
        super(SimpleNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.BatchNorm1d(64),
            nn.Dropout(0.3),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)  # 2 classes for binary classification
        )

    def forward(self, x):
        return self.model(x)

def get_parameters_nn(model):
    """Extract parameters from PyTorch model"""
    return [val.cpu().numpy() for _, val in model.state_dict().items()]

def set_parameters_nn(model, parameters):
    """Set parameters for PyTorch model with proper device handling"""
    # Get the device the model is on
    model_device = next(model.parameters()).device
    params_dict = zip(model.state_dict().keys(), parameters)
    state_dict = {k: torch.tensor(v, device=model_device) for k, v in params_dict}
    model.load_state_dict(state_dict, strict=True)

def train_nn(model, X_train, y_train, epochs=NN_EPOCHS, batch_size=NN_BATCH_SIZE):
    """Train Neural Network model with proper device handling"""
    # Get device from model
    model_device = next(model.parameters()).device

    # Make copies of arrays to ensure they are writable
    X_train_copy = np.array(X_train, copy=True)
    y_train_copy = np.array(y_train, copy=True)

    # Convert to tensors and move to same device as model
    X_tensor = torch.FloatTensor(X_train_copy).to(model_device)
    y_tensor = torch.LongTensor(y_train_copy).to(model_device)

    # Create dataset and dataloader
    dataset = TensorDataset(X_tensor, y_tensor)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Define loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=NN_LEARNING_RATE)

    model.train()
    total_loss = 0

    for epoch in range(epochs):
        epoch_loss = 0
        for batch_X, batch_y in dataloader:
            # Data should already be on correct device
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

        total_loss += epoch_loss / len(dataloader)

    return total_loss / epochs

def evaluate_nn(model, X_test, y_test):
    """Evaluate Neural Network model with proper device handling"""
    model.eval()

    # Get device from model
    model_device = next(model.parameters()).device

    with torch.no_grad():
        X_tensor = torch.FloatTensor(X_test).to(model_device)
        y_tensor = torch.LongTensor(y_test).to(model_device)

        outputs = model(X_tensor)

        # Get predictions from logits
        _, predictions = torch.max(outputs, 1)

        # Calculate metrics
        accuracy = (predictions == y_tensor).float().mean().item()

        # Convert to numpy for sklearn metrics
        y_pred_np = predictions.cpu().numpy()
        y_true_np = y_tensor.cpu().numpy()

        precision = precision_score(y_true_np, y_pred_np, average='weighted', zero_division=0)
        recall = recall_score(y_true_np, y_pred_np, average='weighted', zero_division=0)
        f1 = f1_score(y_true_np, y_pred_np, average='weighted', zero_division=0)

        # Calculate CrossEntropyLoss
        criterion = nn.CrossEntropyLoss()
        loss = criterion(outputs, y_tensor).item()

        return {
            'loss': loss,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        }
print("Neural Network functions defined successfully!")

In [ ]:
# =============================================================================
# NEURAL NETWORK FEDERATED CLIENT
# =============================================================================

class NeuralNetworkClient(fl.client.NumPyClient):
    """Federated Learning Client for Neural Network with CUDA support"""

    def __init__(self, cid, X_train, y_train, X_val, y_val, input_dim):
        self.cid = cid
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.model = SimpleNN(input_dim)

        # Get available device in this process
        try:
            if torch.cuda.is_available():
                self.device = torch.device("cuda")
            else:
                self.device = torch.device("cpu")
        except:
            self.device = torch.device("cpu")

        # Move model to device
        self.model = self.model.to(self.device)

    def get_parameters(self, config):
        """Get model parameters"""
        return get_parameters_nn(self.model)

    def set_parameters(self, parameters):
        """Set model parameters"""
        set_parameters_nn(self.model, parameters)

    def fit(self, parameters, config):
        """Train the model locally"""
        self.set_parameters(parameters)

        # Train the model
        loss = train_nn(self.model, self.X_train, self.y_train)

        # Return updated parameters and training info
        return self.get_parameters(config), len(self.X_train), {"loss": loss}

    def evaluate(self, parameters, config):
        """Evaluate the model locally"""
        self.set_parameters(parameters)

        # Evaluate the model
        metrics = evaluate_nn(self.model, self.X_val, self.y_val)

        return metrics["loss"], len(self.X_val), {
            "accuracy": metrics["accuracy"],
            "precision": metrics["precision"],
            "recall": metrics["recall"],
            "f1": metrics["f1"]
        }

def create_nn_client_fn(client_train_data, client_val_data, input_dim):
    """Create client function for Neural Network"""
    def client_fn(cid: str) -> fl.client.Client:
        cid_int = int(cid)
        X_train, y_train = client_train_data[cid_int]
        X_val, y_val = client_val_data[cid_int]

        return NeuralNetworkClient(cid_int, X_train, y_train, X_val, y_val, input_dim)

    return client_fn

print("Neural Network federated client defined successfully!")

## Neural Network Federated Training Function

This function orchestrates the federated training process for the Neural Network approach, including:
- Strategy configuration (FedAvg, FedProx, FedOpt, FedAdam)
- Server-side evaluation
- Result collection and processing

In [ ]:
# =============================================================================
# NEURAL NETWORK FEDERATED TRAINING FUNCTION
# =============================================================================

def run_nn_federated_learning(client_train_data, client_val_data, input_dim,
                             strategy_name, param_value, param_type):
    """
    Run federated learning with Neural Network

    Args:
        client_train_data: Training data for each client
        client_val_data: Validation data for each client
        input_dim: Input dimension for the model
        strategy_name: FL strategy name
        param_value: Strategy parameter value
        param_type: Parameter type (mu, eta, or None)

    Returns:
        Dictionary containing final metrics
    """

    print(f"Running NN FL with {strategy_name} ({param_type}={param_value})")

    # Create dummy model for initial parameters
    dummy_model = SimpleNN(input_dim)
    try:
        if torch.cuda.is_available():
            dummy_device = torch.device("cuda")
        else:
            dummy_device = torch.device("cpu")
    except:
        dummy_device = torch.device("cpu")

    dummy_model = dummy_model.to(dummy_device)
    initial_parameters = get_parameters_nn(dummy_model)

    # Store results using shared state
    shared_state = {"final_metrics": None, "final_parameters": None}

    def get_evaluate_fn():
        """Create server-side evaluation function"""
        def evaluate(server_round, parameters, config):
            # Evaluate on all clients
            all_metrics = []
            for cid in range(NUM_PARTITIONS):
                model = SimpleNN(input_dim)

                # Get available device
                try:
                    if torch.cuda.is_available():
                        eval_device = torch.device("cuda")
                    else:
                        eval_device = torch.device("cpu")
                except:
                    eval_device = torch.device("cpu")

                model = model.to(eval_device)
                set_parameters_nn(model, parameters)
                X_val, y_val = client_val_data[cid]
                metrics = evaluate_nn(model, X_val, y_val)
                all_metrics.append(metrics)

            # Average metrics across clients
            avg_metrics = pd.DataFrame(all_metrics).mean().to_dict()

            # Store final results
            if server_round == NUM_ROUNDS:
                shared_state["final_metrics"] = avg_metrics.copy()
                shared_state["final_parameters"] = parameters

            return avg_metrics["loss"], {"accuracy": avg_metrics["accuracy"]}

        return evaluate

    # Client function using new API
    def client_fn(context: Context) -> Client:
        partition_id = context.node_config["partition-id"]
        X_train, y_train = client_train_data[partition_id]
        X_val, y_val = client_val_data[partition_id]

        numpy_client = NeuralNetworkClient(partition_id, X_train, y_train, X_val, y_val, input_dim)
        return numpy_client.to_client()

    # Server function using new API
    def server_fn(context: Context) -> ServerAppComponents:
        # Configure strategy
        if strategy_name == "FedAvg":
            strategy = FedAvg(
                min_available_clients=NUM_PARTITIONS,
                evaluate_fn=get_evaluate_fn(),
            )
        elif strategy_name == "FedProx":
            strategy = FedProx(
                proximal_mu=param_value,
                min_available_clients=NUM_PARTITIONS,
                evaluate_fn=get_evaluate_fn(),
            )
        elif strategy_name == "FedAdam":
            strategy = FedAdam(
                initial_parameters=ndarrays_to_parameters(initial_parameters),
                eta=param_value,
                beta_1=0.9,
                beta_2=0.99,
                tau=1e-9,
                min_available_clients=NUM_PARTITIONS,
                evaluate_fn=get_evaluate_fn(),
            )
        elif strategy_name == "FedOpt":
            strategy = FedOpt(
                initial_parameters=ndarrays_to_parameters(initial_parameters),
                eta=param_value,
                beta_1=0.9,
                beta_2=0.99,
                tau=1e-9,
                min_available_clients=NUM_PARTITIONS,
                evaluate_fn=get_evaluate_fn(),
            )
        else:
            raise ValueError(f"Unknown strategy: {strategy_name}")

        return ServerAppComponents(config=ServerConfig(num_rounds=NUM_ROUNDS), strategy=strategy)

    # Create client and server apps
    client = ClientApp(client_fn=client_fn)
    server = ServerApp(server_fn=server_fn)

    # Run federated learning simulation
    try:
        run_simulation(
            client_app=client,
            server_app=server,
            num_supernodes=NUM_PARTITIONS,
            backend_config={"client_resources": {"num_cpus": 2, "num_gpus": 0.0}},
        )

        return shared_state["final_metrics"]

    except Exception as e:
        print(f"Error in NN federated learning: {e}")
        return {"accuracy": 0, "precision": 0, "recall": 0, "f1": 0, "loss": float('inf')}

print("Neural Network federated training function ready!")

## Support Vector Machine Implementation

This section implements Support Vector Machine (SVM) with hinge loss for binary classification using scikit-learn, including:
- SVM model initialization and training
- Parameter serialization for federated learning
- Federated learning client implementation
- Integration with FL strategies

In [ ]:
# =============================================================================
# SVM MODEL DEFINITION AND UTILITIES
# =============================================================================

def initialize_svm_model(n_features):
    """
    Initialize SVM model with hinge loss

    Args:
        n_features: Number of input features

    Returns:
        Initialized SGDClassifier with SVM configuration
    """
    model = SGDClassifier(
        loss="hinge",           # SVM hinge loss
        max_iter=20,           # Local iterations
        tol=None,              # No tolerance for early stopping
        warm_start=True,       # Continue training from previous state
        learning_rate="optimal", # Optimal learning rate
        eta0=0.01,             # Initial learning rate
        random_state=42,       # For reproducibility
    )
    return model

def initialize_svm_model_centralized(n_features):
    """Initialize SVM model with 200 iterations for centralized/local training"""
    model = SGDClassifier(
        loss="hinge",           # SVM hinge loss
        max_iter=200,          # 200 iterations for centralized/local
        tol=None,              # No tolerance for early stopping
        warm_start=True,       # Continue training from previous state
        learning_rate="optimal", # Optimal learning rate
        eta0=0.01,             # Initial learning rate
        random_state=42,       # For reproducibility
    )
    return model

def get_parameters_svm(model):
    """Extract parameters from SVM model"""
    if hasattr(model, 'coef_') and hasattr(model, 'intercept_'):
        return [model.coef_.flatten(), model.intercept_.flatten()]
    else:
        # For untrained models, return zeros
        return [np.zeros(300), np.zeros(1)]  # Assuming 300 features

def set_parameters_svm(model, parameters):
    """Set parameters for SVM model"""
    if len(parameters) >= 2:
        model.coef_ = parameters[0].reshape(1, -1)
        model.intercept_ = parameters[1]
        model.classes_ = np.array([0, 1])  # Binary classification
        # Mark as fitted
        model._expanded_class_weight = [1.0, 1.0]
        model.n_features_in_ = len(parameters[0])

def train_svm(model, X_train, y_train):
    """Train SVM model with error handling"""
    try:
        # Ensure model is properly initialized
        if not hasattr(model, 'classes_'):
            model.partial_fit(X_train[:2], y_train[:2], classes=[0, 1])

        # Continue training
        model.partial_fit(X_train, y_train)
        return True
    except Exception as e:
        print(f"SVM training error: {e}")
        return False

def evaluate_svm(model, X_test, y_test):
    """Evaluate SVM model with comprehensive metrics"""
    try:
        # Get predictions
        y_pred = model.predict(X_test)

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        # Calculate hinge loss
        try:
            decision_scores = model.decision_function(X_test)
            # Manual hinge loss calculation
            hinge_losses = np.maximum(0, 1 - y_test * decision_scores)
            loss = np.mean(hinge_losses)
        except:
            loss = 1.0  # Default loss if calculation fails

        return {
            'loss': loss,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        }
    except Exception as e:
        print(f"SVM evaluation error: {e}")
        return {'loss': 1.0, 'accuracy': 0, 'precision': 0, 'recall': 0, 'f1': 0}

print("SVM utility functions defined successfully!")

In [ ]:
# =============================================================================
# SVM FEDERATED CLIENT
# =============================================================================

class SVMClient(fl.client.NumPyClient):
    """Federated Learning Client for SVM"""

    def __init__(self, cid, X_train, y_train, X_val, y_val, input_dim):
        self.cid = cid
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.model = initialize_svm_model(input_dim)

        # Initialize model with a small batch to set up classes
        if len(self.X_train) >= 2:
            unique_classes = np.unique(self.y_train)
            if len(unique_classes) >= 2:
                # Find indices for each class
                indices = []
                for cls in [0, 1]:
                    cls_indices = np.where(self.y_train == cls)[0]
                    if len(cls_indices) > 0:
                        indices.append(cls_indices[0])

                if len(indices) >= 2:
                    init_X = self.X_train[indices]
                    init_y = self.y_train[indices]
                    self.model.partial_fit(init_X, init_y, classes=[0, 1])

    def get_parameters(self, config):
        """Get model parameters"""
        return get_parameters_svm(self.model)

    def set_parameters(self, parameters):
        """Set model parameters"""
        set_parameters_svm(self.model, parameters)

    def fit(self, parameters, config):
        """Train the model locally"""
        self.set_parameters(parameters)

        # Train the model
        success = train_svm(self.model, self.X_train, self.y_train)

        if success:
            # Evaluate training loss
            train_metrics = evaluate_svm(self.model, self.X_train, self.y_train)
            loss = train_metrics["loss"]
        else:
            loss = 1.0

        return self.get_parameters(config), len(self.X_train), {"loss": loss}

    def evaluate(self, parameters, config):
        """Evaluate the model locally"""
        self.set_parameters(parameters)

        # Evaluate the model
        metrics = evaluate_svm(self.model, self.X_val, self.y_val)

        return metrics["loss"], len(self.X_val), {
            "accuracy": metrics["accuracy"],
            "precision": metrics["precision"],
            "recall": metrics["recall"],
            "f1": metrics["f1"]
        }

def create_svm_client_fn(client_train_data, client_val_data, input_dim):
    """Create client function for SVM"""
    def client_fn(cid: str) -> fl.client.Client:
        cid_int = int(cid)
        X_train, y_train = client_train_data[cid_int]
        X_val, y_val = client_val_data[cid_int]

        return SVMClient(cid_int, X_train, y_train, X_val, y_val, input_dim)

    return client_fn

print("SVM federated client defined successfully!")

## SVM Federated Training Function

This function handles the federated training process for SVM, including:
- Strategy configuration with proper parameter initialization
- Server-side model evaluation
- Result aggregation across clients
- Error handling for distributed training


In [ ]:
# =============================================================================
# SVM FEDERATED TRAINING FUNCTION
# =============================================================================

def run_svm_federated_learning(client_train_data, client_val_data, input_dim,
                              strategy_name, param_value, param_type):
    """
    Run federated learning with SVM

    Args:
        client_train_data: Training data for each client
        client_val_data: Validation data for each client
        input_dim: Input dimension for the model
        strategy_name: FL strategy name
        param_value: Strategy parameter value
        param_type: Parameter type (mu, eta, or None)

    Returns:
        Dictionary containing final metrics
    """

    print(f"Running SVM FL with {strategy_name} ({param_type}={param_value})")

    # Create dummy model for initial parameters
    dummy_model = initialize_svm_model(input_dim)

    # Initialize with some sample data
    if len(client_train_data) > 0:
        X_init, y_init = client_train_data[0]
        if len(X_init) >= 2:
            init_samples = min(5, len(X_init))
            dummy_model.partial_fit(X_init[:init_samples], y_init[:init_samples], classes=[0, 1])

    initial_parameters = get_parameters_svm(dummy_model)

    # Store results using shared state
    shared_state = {"final_metrics": None, "final_parameters": None}

    def get_evaluate_fn():
        """Create server-side evaluation function"""
        def evaluate(server_round, parameters, config):
            # Evaluate on all clients
            all_metrics = []
            for cid in range(NUM_PARTITIONS):
                model = initialize_svm_model(input_dim)
                set_parameters_svm(model, parameters)
                X_val, y_val = client_val_data[cid]
                metrics = evaluate_svm(model, X_val, y_val)
                all_metrics.append(metrics)

            # Average metrics across clients
            avg_metrics = pd.DataFrame(all_metrics).mean().to_dict()

            # Store final results
            if server_round == NUM_ROUNDS:
                shared_state["final_metrics"] = avg_metrics.copy()
                shared_state["final_parameters"] = parameters

            return avg_metrics["loss"], {"accuracy": avg_metrics["accuracy"]}

        return evaluate

    # Client function using new API
    def client_fn(context: Context) -> Client:
        partition_id = context.node_config["partition-id"]
        X_train, y_train = client_train_data[partition_id]
        X_val, y_val = client_val_data[partition_id]

        numpy_client = SVMClient(partition_id, X_train, y_train, X_val, y_val, input_dim)
        return numpy_client.to_client()

    # Server function using new API
    def server_fn(context: Context) -> ServerAppComponents:
        # Configure strategy
        if strategy_name == "FedAvg":
            strategy = FedAvg(
                min_available_clients=NUM_PARTITIONS,
                evaluate_fn=get_evaluate_fn(),
            )
        elif strategy_name == "FedProx":
            strategy = FedProx(
                proximal_mu=param_value,
                min_available_clients=NUM_PARTITIONS,
                evaluate_fn=get_evaluate_fn(),
            )
        elif strategy_name == "FedAdam":
            strategy = FedAdam(
                initial_parameters=ndarrays_to_parameters(initial_parameters),
                eta=param_value,
                beta_1=0.9,
                beta_2=0.99,
                tau=1e-9,
                min_available_clients=NUM_PARTITIONS,
                evaluate_fn=get_evaluate_fn(),
            )
        elif strategy_name == "FedOpt":
            strategy = FedOpt(
                initial_parameters=ndarrays_to_parameters(initial_parameters),
                eta=param_value,
                beta_1=0.9,
                beta_2=0.99,
                tau=1e-9,
                min_available_clients=NUM_PARTITIONS,
                evaluate_fn=get_evaluate_fn(),
            )
        else:
            raise ValueError(f"Unknown strategy: {strategy_name}")

        return ServerAppComponents(config=ServerConfig(num_rounds=NUM_ROUNDS), strategy=strategy)

    # Create client and server apps
    client = ClientApp(client_fn=client_fn)
    server = ServerApp(server_fn=server_fn)

    # Run federated learning simulation
    try:
        run_simulation(
            client_app=client,
            server_app=server,
            num_supernodes=NUM_PARTITIONS,
            backend_config={"client_resources": {"num_cpus": 2, "num_gpus": 0.0}},
        )

        return shared_state["final_metrics"]

    except Exception as e:
        print(f"Error in SVM federated learning: {e}")
        return {"accuracy": 0, "precision": 0, "recall": 0, "f1": 0, "loss": float('inf')}

print("SVM federated training function ready!")

## Logistic Regression Implementation

This section implements Logistic Regression for binary classification using scikit-learn's SGDClassifier with log loss, including:
- Logistic regression model initialization and training
- Parameter management for federated learning
- Federated learning client implementation
- Integration with various FL strategies

In [ ]:
# =============================================================================
# LOGISTIC REGRESSION MODEL DEFINITION AND UTILITIES
# =============================================================================

def initialize_lr_model(n_features):
    """
    Initialize Logistic Regression model with log loss

    Args:
        n_features: Number of input features

    Returns:
        Initialized SGDClassifier with Logistic Regression configuration
    """
    model = SGDClassifier(
        loss="log_loss",        # Logistic regression loss
        max_iter=20,           # Local iterations
        tol=None,              # No tolerance for early stopping
        warm_start=True,       # Continue training from previous state
        learning_rate="optimal", # Optimal learning rate
        eta0=0.01,             # Initial learning rate
        random_state=42,       # For reproducibility
    )
    return model

def initialize_lr_model_centralized(n_features):
    """Initialize Logistic Regression model with 200 iterations for centralized/local training"""
    model = SGDClassifier(
        loss="log_loss",        # Logistic regression loss
        max_iter=200,          # 200 iterations for centralized/local
        tol=None,              # No tolerance for early stopping
        warm_start=True,       # Continue training from previous state
        learning_rate="optimal", # Optimal learning rate
        eta0=0.01,             # Initial learning rate
        random_state=42,       # For reproducibility
    )
    return model

def get_parameters_lr(model):
    """Extract parameters from Logistic Regression model"""
    if hasattr(model, 'coef_') and hasattr(model, 'intercept_'):
        return [model.coef_.flatten(), model.intercept_.flatten()]
    else:
        # For untrained models, return zeros
        return [np.zeros(300), np.zeros(1)]  # Assuming 300 features

def set_parameters_lr(model, parameters):
    """Set parameters for Logistic Regression model"""
    if len(parameters) >= 2:
        model.coef_ = parameters[0].reshape(1, -1)
        model.intercept_ = parameters[1]
        model.classes_ = np.array([0, 1])  # Binary classification
        # Mark as fitted
        model._expanded_class_weight = [1.0, 1.0]
        model.n_features_in_ = len(parameters[0])

def train_lr(model, X_train, y_train):
    """Train Logistic Regression model with error handling"""
    try:
        # Ensure model is properly initialized
        if not hasattr(model, 'classes_'):
            model.partial_fit(X_train[:2], y_train[:2], classes=[0, 1])

        # Continue training
        model.partial_fit(X_train, y_train)
        return True
    except Exception as e:
        print(f"LR training error: {e}")
        return False

def evaluate_lr(model, X_test, y_test):
    """Evaluate Logistic Regression model with comprehensive metrics"""
    try:
        # Get predictions
        y_pred = model.predict(X_test)

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)

        # Calculate log loss
        try:
            # Get probability predictions
            y_proba = model.predict_proba(X_test)
            # Calculate log loss manually to avoid sklearn warnings
            epsilon = 1e-15  # Small value to avoid log(0)
            y_proba = np.clip(y_proba, epsilon, 1 - epsilon)

            # Convert y_test to one-hot for log loss calculation
            n_classes = y_proba.shape[1]
            y_one_hot = np.eye(n_classes)[y_test]

            loss = -np.mean(np.sum(y_one_hot * np.log(y_proba), axis=1))
        except:
            loss = 1.0  # Default loss if calculation fails

        return {
            'loss': loss,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1': f1
        }
    except Exception as e:
        print(f"LR evaluation error: {e}")
        return {'loss': 1.0, 'accuracy': 0, 'precision': 0, 'recall': 0, 'f1': 0}

print("Logistic Regression utility functions defined successfully!")

In [ ]:
# =============================================================================
# LOGISTIC REGRESSION FEDERATED CLIENT
# =============================================================================

class LogisticRegressionClient(fl.client.NumPyClient):
    """Federated Learning Client for Logistic Regression"""

    def __init__(self, cid, X_train, y_train, X_val, y_val, input_dim):
        self.cid = cid
        self.X_train = X_train
        self.y_train = y_train
        self.X_val = X_val
        self.y_val = y_val
        self.model = initialize_lr_model(input_dim)

        # Initialize model with a small batch to set up classes
        if len(self.X_train) >= 2:
            unique_classes = np.unique(self.y_train)
            if len(unique_classes) >= 2:
                # Find indices for each class
                indices = []
                for cls in [0, 1]:
                    cls_indices = np.where(self.y_train == cls)[0]
                    if len(cls_indices) > 0:
                        indices.append(cls_indices[0])

                if len(indices) >= 2:
                    init_X = self.X_train[indices]
                    init_y = self.y_train[indices]
                    self.model.partial_fit(init_X, init_y, classes=[0, 1])

    def get_parameters(self, config):
        """Get model parameters"""
        return get_parameters_lr(self.model)

    def set_parameters(self, parameters):
        """Set model parameters"""
        set_parameters_lr(self.model, parameters)

    def fit(self, parameters, config):
        """Train the model locally"""
        self.set_parameters(parameters)

        # Train the model
        success = train_lr(self.model, self.X_train, self.y_train)

        if success:
            # Evaluate training loss
            train_metrics = evaluate_lr(self.model, self.X_train, self.y_train)
            loss = train_metrics["loss"]
        else:
            loss = 1.0

        return self.get_parameters(config), len(self.X_train), {"loss": loss}

    def evaluate(self, parameters, config):
        """Evaluate the model locally"""
        self.set_parameters(parameters)

        # Evaluate the model
        metrics = evaluate_lr(self.model, self.X_val, self.y_val)

        return metrics["loss"], len(self.X_val), {
            "accuracy": metrics["accuracy"],
            "precision": metrics["precision"],
            "recall": metrics["recall"],
            "f1": metrics["f1"]
        }

def create_lr_client_fn(client_train_data, client_val_data, input_dim):
    """Create client function for Logistic Regression"""
    def client_fn(cid: str) -> fl.client.Client:
        cid_int = int(cid)
        X_train, y_train = client_train_data[cid_int]
        X_val, y_val = client_val_data[cid_int]

        return LogisticRegressionClient(cid_int, X_train, y_train, X_val, y_val, input_dim)

    return client_fn

print("Logistic Regression federated client defined successfully!")

## Logistic Regression Federated Training Function

This function orchestrates the federated training process for Logistic Regression, including:
- Proper model initialization with sample data
- Strategy configuration for different FL algorithms
- Server-side evaluation and metrics aggregation
- Robust error handling for distributed training

In [ ]:
# =============================================================================
# LOGISTIC REGRESSION FEDERATED TRAINING FUNCTION
# =============================================================================

def run_lr_federated_learning(client_train_data, client_val_data, input_dim,
                             strategy_name, param_value, param_type):
    """
    Run federated learning with Logistic Regression

    Args:
        client_train_data: Training data for each client
        client_val_data: Validation data for each client
        input_dim: Input dimension for the model
        strategy_name: FL strategy name
        param_value: Strategy parameter value
        param_type: Parameter type (mu, eta, or None)

    Returns:
        Dictionary containing final metrics
    """

    print(f"Running LR FL with {strategy_name} ({param_type}={param_value})")

    # Create dummy model for initial parameters
    dummy_model = initialize_lr_model(input_dim)

    # Initialize with some sample data
    if len(client_train_data) > 0:
        X_init, y_init = client_train_data[0]
        if len(X_init) >= 2:
            init_samples = min(5, len(X_init))
            dummy_model.partial_fit(X_init[:init_samples], y_init[:init_samples], classes=[0, 1])

    initial_parameters = get_parameters_lr(dummy_model)

    # Store results using shared state
    shared_state = {"final_metrics": None, "final_parameters": None}

    def get_evaluate_fn():
        """Create server-side evaluation function"""
        def evaluate(server_round, parameters, config):
            # Evaluate on all clients
            all_metrics = []
            for cid in range(NUM_PARTITIONS):
                model = initialize_lr_model(input_dim)
                set_parameters_lr(model, parameters)
                X_val, y_val = client_val_data[cid]
                metrics = evaluate_lr(model, X_val, y_val)
                all_metrics.append(metrics)

            # Average metrics across clients
            avg_metrics = pd.DataFrame(all_metrics).mean().to_dict()

            # Store final results
            if server_round == NUM_ROUNDS:
                shared_state["final_metrics"] = avg_metrics.copy()
                shared_state["final_parameters"] = parameters

            return avg_metrics["loss"], {"accuracy": avg_metrics["accuracy"]}

        return evaluate

    # Client function using new API
    def client_fn(context: Context) -> Client:
        partition_id = context.node_config["partition-id"]
        X_train, y_train = client_train_data[partition_id]
        X_val, y_val = client_val_data[partition_id]

        numpy_client = LogisticRegressionClient(partition_id, X_train, y_train, X_val, y_val, input_dim)
        return numpy_client.to_client()

    # Server function using new API
    def server_fn(context: Context) -> ServerAppComponents:
        # Configure strategy
        if strategy_name == "FedAvg":
            strategy = FedAvg(
                min_available_clients=NUM_PARTITIONS,
                evaluate_fn=get_evaluate_fn(),
            )
        elif strategy_name == "FedProx":
            strategy = FedProx(
                proximal_mu=param_value,
                min_available_clients=NUM_PARTITIONS,
                evaluate_fn=get_evaluate_fn(),
            )
        elif strategy_name == "FedAdam":
            strategy = FedAdam(
                initial_parameters=ndarrays_to_parameters(initial_parameters),
                eta=param_value,
                beta_1=0.9,
                beta_2=0.99,
                tau=1e-9,
                min_available_clients=NUM_PARTITIONS,
                evaluate_fn=get_evaluate_fn(),
            )
        elif strategy_name == "FedOpt":
            strategy = FedOpt(
                initial_parameters=ndarrays_to_parameters(initial_parameters),
                eta=param_value,
                beta_1=0.9,
                beta_2=0.99,
                tau=1e-9,
                min_available_clients=NUM_PARTITIONS,
                evaluate_fn=get_evaluate_fn(),
            )
        else:
            raise ValueError(f"Unknown strategy: {strategy_name}")

        return ServerAppComponents(config=ServerConfig(num_rounds=NUM_ROUNDS), strategy=strategy)

    # Create client and server apps
    client = ClientApp(client_fn=client_fn)
    server = ServerApp(server_fn=server_fn)

    # Run federated learning simulation
    try:
        run_simulation(
            client_app=client,
            server_app=server,
            num_supernodes=NUM_PARTITIONS,
            backend_config={"client_resources": {"num_cpus": 2, "num_gpus": 0.0}},
        )

        return shared_state["final_metrics"]

    except Exception as e:
        print(f"Error in LR federated learning: {e}")
        return {"accuracy": 0, "precision": 0, "recall": 0, "f1": 0, "loss": float('inf')}

print("Logistic Regression federated training function ready!")

## Main Experiment Framework

This section orchestrates the complete federated learning comparison experiment, including:
- Cross-validation setup with multiple folds
- Federated learning experiments for all three approaches
- Centralized and local training baselines
- Results collection and analysis
- Performance visualization and comparison

In [ ]:
# =============================================================================
# CENTRALIZED AND LOCAL TRAINING BASELINES
# =============================================================================

def run_centralized_training(X_train_central, y_train_central, X_test_central, y_test_central, model_type):
    """
    Run centralized training for comparison baseline

    Args:
        X_train_central: Centralized training features
        y_train_central: Centralized training labels
        X_test_central: Centralized test features
        y_test_central: Centralized test labels
        model_type: Type of model ('nn', 'svm', 'lr')

    Returns:
        Dictionary containing evaluation metrics
    """

    print(f"Running centralized {model_type.upper()} training...")

    try:
        if model_type == 'nn':
            # Neural Network centralized training with proper device handling
            model = SimpleNN(X_train_central.shape[1])

            # Get available device
            try:
                if torch.cuda.is_available():
                    model_device = torch.device("cuda")
                else:
                    model_device = torch.device("cpu")
            except:
                model_device = torch.device("cpu")

            model = model.to(model_device)
            train_nn(model, X_train_central, y_train_central, epochs=NN_EPOCHS*10)
            metrics = evaluate_nn(model, X_test_central, y_test_central)

        elif model_type == 'svm':
            # SVM centralized training (unchanged)
            model = initialize_svm_model_centralized(X_train_central.shape[1])
            model.fit(X_train_central, y_train_central)
            metrics = evaluate_svm(model, X_test_central, y_test_central)

        elif model_type == 'lr':
            # Logistic Regression centralized training (unchanged)
            model = initialize_lr_model_centralized(X_train_central.shape[1])
            model.fit(X_train_central, y_train_central)
            metrics = evaluate_lr(model, X_test_central, y_test_central)

        return metrics

    except Exception as e:
        print(f"Error in centralized {model_type} training: {e}")
        return {"accuracy": 0, "precision": 0, "recall": 0, "f1": 0, "loss": float('inf')}

def run_local_training(client_train_data, client_val_data, model_type):
    """
    Run local training on each client separately

    Args:
        client_train_data: Training data for each client
        client_val_data: Validation data for each client
        model_type: Type of model ('nn', 'svm', 'lr')

    Returns:
        Dictionary containing averaged evaluation metrics across clients
    """

    print(f"Running local {model_type.upper()} training...")

    all_metrics = []

    for cid in range(NUM_PARTITIONS):
        try:
            X_train, y_train = client_train_data[cid]
            X_val, y_val = client_val_data[cid]

            if model_type == 'nn':
                # Neural Network local training with proper device handling
                model = SimpleNN(X_train.shape[1])

                # Get available device
                try:
                    if torch.cuda.is_available():
                        model_device = torch.device("cuda")
                    else:
                        model_device = torch.device("cpu")
                except:
                    model_device = torch.device("cpu")

                model = model.to(model_device)
                train_nn(model, X_train, y_train, epochs=NN_EPOCHS*10)
                metrics = evaluate_nn(model, X_val, y_val)

            elif model_type == 'svm':
                # SVM local training (unchanged)
                model = initialize_svm_model_centralized(X_train.shape[1])
                model.fit(X_train, y_train)
                metrics = evaluate_svm(model, X_val, y_val)

            elif model_type == 'lr':
                # Logistic Regression local training (unchanged)
                model = initialize_lr_model_centralized(X_train.shape[1])
                model.fit(X_train, y_train)
                metrics = evaluate_lr(model, X_val, y_val)

            all_metrics.append(metrics)

        except Exception as e:
            print(f"Error in local {model_type} training for client {cid}: {e}")
            all_metrics.append({"accuracy": 0, "precision": 0, "recall": 0, "f1": 0, "loss": float('inf')})

    # Average metrics across all clients
    avg_metrics = pd.DataFrame(all_metrics).mean().to_dict()
    return avg_metrics

print("Baseline training functions defined successfully!")

In [ ]:
# =============================================================================
# MAIN EXPERIMENT ORCHESTRATION
# =============================================================================

def run_complete_experiment(X, y, model_type='nn', apply_splines=True, custom_hyperparams=None):
    """
    Run complete federated learning experiment with cross-validation

    Args:
        X: Feature matrix
        y: Labels
        model_type: Type of model ('nn', 'svm', 'lr')
        apply_splines: Whether to apply B-spline preprocessing
        custom_hyperparams: Optional custom hyperparameter configuration

    Returns:
        Dictionary containing all experimental results
    """

    print(f"\n{'='*80}")
    print(f"STARTING COMPLETE EXPERIMENT: {model_type.upper()}")

    # Get model-specific strategies
    strategies = configure_model_hyperparameters(model_type, custom_hyperparams)

    print(f"Model-specific strategies loaded:")
    for strategy_name, params in strategies.items():
        param_type = list(params.keys())[0]
        param_values = list(params.values())[0]
        print(f"  {strategy_name}: {param_type} = {param_values}")

    print(f"{'='*80}")

    # Results storage
    all_results = {
        'federated': {},
        'centralized': [],
        'local': []
    }

    # Initialize federated results for each strategy
    for strategy_name in strategies.keys():
        all_results['federated'][strategy_name] = {}

    # Cross-validation loop
    for fold_idx in range(NUM_FOLDS):
        print(f"\n--- FOLD {fold_idx + 1}/{NUM_FOLDS} ---")

        # Create federated data partitions
        data_partitions = create_federated_data_partitions(X, y, fold_idx, apply_splines)

        client_train_data = data_partitions['client_train_data']
        client_val_data = data_partitions['client_val_data']
        centralized_train = data_partitions['centralized_train']
        centralized_test = data_partitions['centralized_test']
        input_dim = data_partitions['input_dim']

        # Run centralized training
        print("Running centralized baseline...")
        centralized_metrics = run_centralized_training(
            centralized_train[0], centralized_train[1],
            centralized_test[0], centralized_test[1],
            model_type
        )
        all_results['centralized'].append(centralized_metrics)

        # Run local training
        print("Running local baseline...")
        local_metrics = run_local_training(client_train_data, client_val_data, model_type)
        all_results['local'].append(local_metrics)

        # Run federated learning for each strategy
        for strategy_name, params in strategies.items():
            print(f"\nTesting {strategy_name}...")

            # Get parameter values for this strategy
            if "mu_values" in params:
                param_values = params["mu_values"]
                param_type = "mu"
            elif "eta_values" in params:
                param_values = params["eta_values"]
                param_type = "eta"
            else:
                param_values = params["no_values"]
                param_type = "none"

            # Initialize results for this strategy if not exists
            if strategy_name not in all_results['federated']:
                all_results['federated'][strategy_name] = {}

            # Test each parameter value
            for param_value in param_values:
                print(f"  Testing {param_type}={param_value}")

                # Initialize results for this parameter if not exists
                param_key = f"{param_type}_{param_value}"
                if param_key not in all_results['federated'][strategy_name]:
                    all_results['federated'][strategy_name][param_key] = []

                # Run federated learning
                if model_type == 'nn':
                    fl_metrics = run_nn_federated_learning(
                        client_train_data, client_val_data, input_dim,
                        strategy_name, param_value, param_type
                    )
                elif model_type == 'svm':
                    fl_metrics = run_svm_federated_learning(
                        client_train_data, client_val_data, input_dim,
                        strategy_name, param_value, param_type
                    )
                elif model_type == 'lr':
                    fl_metrics = run_lr_federated_learning(
                        client_train_data, client_val_data, input_dim,
                        strategy_name, param_value, param_type
                    )

                all_results['federated'][strategy_name][param_key].append(fl_metrics)

        print(f"Fold {fold_idx + 1} completed!")

    return all_results

print("Main experiment orchestration function ready!")

## Results Analysis and Visualization

This section provides comprehensive analysis of the experimental results, including:
- Statistical summary of performance metrics
- Comparison across different FL strategies
- Visualization of results with plots and tables
- Best hyperparameter identification

In [ ]:
# =============================================================================
# RESULTS ANALYSIS AND VISUALIZATION
# =============================================================================

def analyze_results(results, model_type):
    """
    Analyze and summarize experimental results

    Args:
        results: Dictionary containing all experimental results
        model_type: Type of model ('nn', 'svm', 'lr')

    Returns:
        Dictionary containing analysis summary
    """

    print(f"\n{'='*60}")
    print(f"RESULTS ANALYSIS: {model_type.upper()}")
    print(f"{'='*60}")

    analysis = {
        'summary_stats': {},
        'best_configs': {},
        'comparison_table': None
    }

    # Analyze centralized results
    if results['centralized']:
        centralized_df = pd.DataFrame(results['centralized'])
        analysis['summary_stats']['centralized'] = {
            'accuracy_mean': centralized_df['accuracy'].mean(),
            'accuracy_std': centralized_df['accuracy'].std(),
            'f1_mean': centralized_df['f1'].mean(),
            'f1_std': centralized_df['f1'].std()
        }
        print(f"Centralized - Accuracy: {centralized_df['accuracy'].mean():.4f} ± {centralized_df['accuracy'].std():.4f}")
        print(f"Centralized - F1: {centralized_df['f1'].mean():.4f} ± {centralized_df['f1'].std():.4f}")

    # Analyze local results
    if results['local']:
        local_df = pd.DataFrame(results['local'])
        analysis['summary_stats']['local'] = {
            'accuracy_mean': local_df['accuracy'].mean(),
            'accuracy_std': local_df['accuracy'].std(),
            'f1_mean': local_df['f1'].mean(),
            'f1_std': local_df['f1'].std()
        }
        print(f"Local - Accuracy: {local_df['accuracy'].mean():.4f} ± {local_df['accuracy'].std():.4f}")
        print(f"Local - F1: {local_df['f1'].mean():.4f} ± {local_df['f1'].std():.4f}")

    # Analyze federated results
    print(f"\nFederated Learning Results:")
    best_accuracy = 0
    best_config = None

    for strategy_name, strategy_results in results['federated'].items():
        print(f"\n{strategy_name}:")
        analysis['summary_stats'][strategy_name] = {}

        for param_key, param_results in strategy_results.items():
            if param_results:  # Check if results exist
                param_df = pd.DataFrame(param_results)
                acc_mean = param_df['accuracy'].mean()
                acc_std = param_df['accuracy'].std()
                f1_mean = param_df['f1'].mean()
                f1_std = param_df['f1'].std()

                analysis['summary_stats'][strategy_name][param_key] = {
                    'accuracy_mean': acc_mean,
                    'accuracy_std': acc_std,
                    'f1_mean': f1_mean,
                    'f1_std': f1_std
                }

                print(f"  {param_key}: Acc={acc_mean:.4f}±{acc_std:.4f}, F1={f1_mean:.4f}±{f1_std:.4f}")

                # Track best configuration
                if acc_mean > best_accuracy:
                    best_accuracy = acc_mean
                    best_config = f"{strategy_name}_{param_key}"

    analysis['best_configs']['accuracy'] = best_config
    print(f"\nBest FL Configuration: {best_config} (Accuracy: {best_accuracy:.4f})")

    return analysis

def create_comparison_table(results, model_type):
    """Create a comparison table of all approaches"""

    print(f"\n{'='*60}")
    print(f"PERFORMANCE COMPARISON TABLE: {model_type.upper()}")
    print(f"{'='*60}")

    comparison_data = []

    # Add centralized results
    if results['centralized']:
        centralized_df = pd.DataFrame(results['centralized'])
        comparison_data.append({
            'Approach': 'Centralized',
            'Strategy': 'N/A',
            'Parameter': 'N/A',
            'Accuracy': f"{centralized_df['accuracy'].mean():.4f} ± {centralized_df['accuracy'].std():.4f}",
            'F1': f"{centralized_df['f1'].mean():.4f} ± {centralized_df['f1'].std():.4f}",
            'Precision': f"{centralized_df['precision'].mean():.4f} ± {centralized_df['precision'].std():.4f}",
            'Recall': f"{centralized_df['recall'].mean():.4f} ± {centralized_df['recall'].std():.4f}"
        })

    # Add local results
    if results['local']:
        local_df = pd.DataFrame(results['local'])
        comparison_data.append({
            'Approach': 'Local',
            'Strategy': 'N/A',
            'Parameter': 'N/A',
            'Accuracy': f"{local_df['accuracy'].mean():.4f} ± {local_df['accuracy'].std():.4f}",
            'F1': f"{local_df['f1'].mean():.4f} ± {local_df['f1'].std():.4f}",
            'Precision': f"{local_df['precision'].mean():.4f} ± {local_df['precision'].std():.4f}",
            'Recall': f"{local_df['recall'].mean():.4f} ± {local_df['recall'].std():.4f}"
        })

    # Add federated results
    for strategy_name, strategy_results in results['federated'].items():
        for param_key, param_results in strategy_results.items():
            if param_results:
                param_df = pd.DataFrame(param_results)
                comparison_data.append({
                    'Approach': 'Federated',
                    'Strategy': strategy_name,
                    'Parameter': param_key,
                    'Accuracy': f"{param_df['accuracy'].mean():.4f} ± {param_df['accuracy'].std():.4f}",
                    'F1': f"{param_df['f1'].mean():.4f} ± {param_df['f1'].std():.4f}",
                    'Precision': f"{param_df['precision'].mean():.4f} ± {param_df['precision'].std():.4f}",
                    'Recall': f"{param_df['recall'].mean():.4f} ± {param_df['recall'].std():.4f}"
                })

    comparison_df = pd.DataFrame(comparison_data)
    print(comparison_df.to_string(index=False))

    return comparison_df

print("Results analysis functions ready!")

## Complete Experiment Execution

This final section executes all three federated learning approaches and generates a comprehensive comparison, including:
- Sequential execution of NN, SVM, and LR experiments
- Cross-approach performance comparison
- Visualization of results
- Summary statistics and insights

In [ ]:
# =============================================================================
# VISUALIZATION FUNCTIONS WITH BOXPLOTS AND MEANS
# =============================================================================

def create_performance_boxplots(all_model_results):
    """
    Create comprehensive performance visualization with boxplots and means

    Args:
        all_model_results: Dictionary containing results for all models
    """

    print("Creating performance boxplot visualizations...")

    # Set up the plotting style
    plt.style.use('default')
    sns.set_palette("husl")

    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('Federated Learning Performance Comparison (Boxplots with Means)', fontsize=16, fontweight='bold')

    # Prepare data for plotting - collect all individual fold results
    plot_data = []

    for model_type, results in all_model_results.items():
        # Centralized results - individual fold results
        if results['centralized']:
            for fold_result in results['centralized']:
                plot_data.append({
                    'Model': model_type.upper(),
                    'Approach': 'Centralized',
                    'Strategy': 'N/A',
                    'Accuracy': fold_result['accuracy'],
                    'F1': fold_result['f1'],
                    'Precision': fold_result['precision'],
                    'Recall': fold_result['recall']
                })

        # Local results - individual fold results
        if results['local']:
            for fold_result in results['local']:
                plot_data.append({
                    'Model': model_type.upper(),
                    'Approach': 'Local',
                    'Strategy': 'N/A',
                    'Accuracy': fold_result['accuracy'],
                    'F1': fold_result['f1'],
                    'Precision': fold_result['precision'],
                    'Recall': fold_result['recall']
                })

        # Federated results - get best performing configuration for each strategy
        for strategy_name, strategy_results in results['federated'].items():
            best_acc = 0
            best_param_key = None

            # Find best parameter configuration
            for param_key, param_results in strategy_results.items():
                if param_results:
                    param_df = pd.DataFrame(param_results)
                    acc = param_df['accuracy'].mean()
                    if acc > best_acc:
                        best_acc = acc
                        best_param_key = param_key

            # Add individual fold results for best configuration
            if best_param_key and strategy_results[best_param_key]:
                for fold_result in strategy_results[best_param_key]:
                    plot_data.append({
                        'Model': model_type.upper(),
                        'Approach': 'Federated',
                        'Strategy': strategy_name,
                        'Accuracy': fold_result['accuracy'],
                        'F1': fold_result['f1'],
                        'Precision': fold_result['precision'],
                        'Recall': fold_result['recall']
                    })

    plot_df = pd.DataFrame(plot_data)

    # Create a combined approach-strategy column for better visualization
    plot_df['Approach_Strategy'] = plot_df.apply(
        lambda row: row['Approach'] if row['Approach'] != 'Federated'
        else f"FL-{row['Strategy']}", axis=1
    )

    # Function to add means to boxplot
    def add_means_to_boxplot(ax, data, x_col, y_col, hue_col):
        """Add mean points to boxplot with correct positioning"""

        # Calculate means
        means = data.groupby([x_col, hue_col])[y_col].mean().reset_index()

        # Get unique categories
        x_categories = data[x_col].unique()
        hue_categories = data[hue_col].unique()

        # Get the positions of the boxes
        n_hue = len(hue_categories)

        for i, x_cat in enumerate(x_categories):
            for j, hue_cat in enumerate(hue_categories):
                # Find the mean for this combination
                mean_val = means[(means[x_col] == x_cat) & (means[hue_col] == hue_cat)][y_col]

                if not mean_val.empty:
                    # Calculate x position - this matches seaborn's positioning
                    if n_hue > 1:
                        # Multiple hue categories - spread them around the main position
                        width = 0.8 / n_hue
                        x_pos = i + (j - (n_hue - 1) / 2) * width
                    else:
                        # Single hue category - center on main position
                        x_pos = i

                    # Add mean point
                    ax.scatter(x_pos, mean_val.iloc[0], color='red', s=60, marker='D',
                             zorder=10, edgecolors='darkred', linewidth=1)

    # Plot 1: Accuracy boxplot with means
    sns.boxplot(data=plot_df, x='Model', y='Accuracy', hue='Approach_Strategy', ax=axes[0,0])
    add_means_to_boxplot(axes[0,0], plot_df, 'Model', 'Accuracy', 'Approach_Strategy')
    axes[0,0].set_title('Accuracy Distribution')
    axes[0,0].set_ylim(0, 1)
    axes[0,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    # Plot 2: F1 Score boxplot with means
    sns.boxplot(data=plot_df, x='Model', y='F1', hue='Approach_Strategy', ax=axes[0,1])
    add_means_to_boxplot(axes[0,1], plot_df, 'Model', 'F1', 'Approach_Strategy')
    axes[0,1].set_title('F1 Score Distribution')
    axes[0,1].set_ylim(0, 1)
    axes[0,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    # Plot 3: Precision boxplot with means
    sns.boxplot(data=plot_df, x='Model', y='Precision', hue='Approach_Strategy', ax=axes[1,0])
    add_means_to_boxplot(axes[1,0], plot_df, 'Model', 'Precision', 'Approach_Strategy')
    axes[1,0].set_title('Precision Distribution')
    axes[1,0].set_ylim(0, 1)
    axes[1,0].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    # Plot 4: Recall boxplot with means
    sns.boxplot(data=plot_df, x='Model', y='Recall', hue='Approach_Strategy', ax=axes[1,1])
    add_means_to_boxplot(axes[1,1], plot_df, 'Model', 'Recall', 'Approach_Strategy')
    axes[1,1].set_title('Recall Distribution')
    axes[1,1].set_ylim(0, 1)
    axes[1,1].legend(bbox_to_anchor=(1.05, 1), loc='upper left')

    plt.tight_layout()
    plt.show()

    # Print summary statistics
    print("\nSummary Statistics (Mean ± Std):")
    summary_stats = plot_df.groupby(['Model', 'Approach_Strategy']).agg({
        'Accuracy': ['mean', 'std'],
        'F1': ['mean', 'std'],
        'Precision': ['mean', 'std'],
        'Recall': ['mean', 'std']
    }).round(4)

    print(summary_stats)

    return plot_df

def create_strategy_comparison_boxplot(all_model_results):
    """Create detailed boxplot comparison of federated learning strategies"""

    print("Creating federated learning strategy comparison boxplots...")

    fig, axes = plt.subplots(1, 3, figsize=(20, 6))
    fig.suptitle('Federated Learning Strategy Performance Distribution', fontsize=16, fontweight='bold')

    model_types = ['nn', 'svm', 'lr']

    for idx, model_type in enumerate(model_types):
        if model_type in all_model_results:
            results = all_model_results[model_type]

            strategy_data = []

            for strategy_name, strategy_results in results['federated'].items():
                for param_key, param_results in strategy_results.items():
                    if param_results:
                        # Add all individual fold results
                        for fold_result in param_results:
                            strategy_data.append({
                                'Strategy_Param': f"{strategy_name}\n({param_key})",
                                'Strategy': strategy_name,
                                'Accuracy': fold_result['accuracy'],
                                'F1': fold_result['f1']
                            })

            if strategy_data:
                strategy_df = pd.DataFrame(strategy_data)

                # Create boxplot
                box_plot = sns.boxplot(data=strategy_df, x='Strategy_Param', y='Accuracy', ax=axes[idx])

                # Add means with proper positioning
                means = strategy_df.groupby('Strategy_Param')['Accuracy'].mean()

                # Get the positions of the boxes from the boxplot
                box_positions = [box.get_x() + box.get_width()/2 for box in box_plot.patches[::len(means)]]

                # If we can't get positions from patches, use simple indexing
                if len(box_positions) != len(means):
                    box_positions = range(len(means))

                # Add mean points
                for pos, (param, mean_val) in zip(box_positions, means.items()):
                    axes[idx].scatter(pos, mean_val, color='red', s=80, marker='D',
                                    zorder=10, edgecolors='darkred', linewidth=1)

                    # Add mean value labels
                    axes[idx].text(pos, mean_val + 0.02, f'{mean_val:.3f}',
                                 ha='center', va='bottom', fontsize=9,
                                 bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8))

                axes[idx].set_title(f'{model_type.upper()} Model')
                axes[idx].set_ylabel('Accuracy')
                axes[idx].set_xlabel('Strategy Configuration')
                axes[idx].tick_params(axis='x', rotation=45)
                axes[idx].set_ylim(0, 1)

    plt.tight_layout()
    plt.show()

print("Visualization functions with boxplots and means ready!")

In [ ]:
# =============================================================================
# COMPREHENSIVE RESULTS SUMMARY
# =============================================================================

def generate_comprehensive_summary(all_model_results):
    """
    Generate a comprehensive summary of all experimental results

    Args:
        all_model_results: Dictionary containing results for all models

    Returns:
        Dictionary containing comprehensive analysis
    """

    print(f"\n{'='*80}")
    print("COMPREHENSIVE EXPERIMENTAL SUMMARY")
    print(f"{'='*80}")

    summary = {
        'overall_best': {},
        'model_rankings': {},
        'strategy_performance': {},
        'insights': []
    }

    # Find overall best performance
    best_accuracy = 0
    best_f1 = 0
    best_model_acc = None
    best_model_f1 = None

    print("\n1. OVERALL BEST PERFORMANCE:")
    print("-" * 40)

    for model_type, results in all_model_results.items():
        print(f"\n{model_type.upper()} Model:")

        model_best_acc = 0
        model_best_f1 = 0
        model_best_config_acc = None
        model_best_config_f1 = None

        # Check all approaches for this model
        approaches = ['centralized', 'local', 'federated']

        for approach in approaches:
            if approach in results and results[approach]:
                if approach == 'federated':
                    # Check all federated strategies
                    for strategy_name, strategy_results in results[approach].items():
                        for param_key, param_results in strategy_results.items():
                            if param_results:
                                param_df = pd.DataFrame(param_results)
                                acc = param_df['accuracy'].mean()
                                f1 = param_df['f1'].mean()

                                config_name = f"{approach}_{strategy_name}_{param_key}"

                                if acc > model_best_acc:
                                    model_best_acc = acc
                                    model_best_config_acc = config_name

                                if f1 > model_best_f1:
                                    model_best_f1 = f1
                                    model_best_config_f1 = config_name
                else:
                    # Centralized or local
                    approach_df = pd.DataFrame(results[approach])
                    acc = approach_df['accuracy'].mean()
                    f1 = approach_df['f1'].mean()

                    if acc > model_best_acc:
                        model_best_acc = acc
                        model_best_config_acc = approach

                    if f1 > model_best_f1:
                        model_best_f1 = f1
                        model_best_config_f1 = approach

        print(f"  Best Accuracy: {model_best_acc:.4f} ({model_best_config_acc})")
        print(f"  Best F1: {model_best_f1:.4f} ({model_best_config_f1})")

        # Update overall best
        if model_best_acc > best_accuracy:
            best_accuracy = model_best_acc
            best_model_acc = f"{model_type}_{model_best_config_acc}"

        if model_best_f1 > best_f1:
            best_f1 = model_best_f1
            best_model_f1 = f"{model_type}_{model_best_config_f1}"

    summary['overall_best']['accuracy'] = {'value': best_accuracy, 'config': best_model_acc}
    summary['overall_best']['f1'] = {'value': best_f1, 'config': best_model_f1}

    print(f"\nOVERALL BEST ACCURACY: {best_accuracy:.4f} ({best_model_acc})")
    print(f"OVERALL BEST F1: {best_f1:.4f} ({best_model_f1})")

    # Strategy performance analysis
    print(f"\n2. FEDERATED LEARNING STRATEGY ANALYSIS:")
    print("-" * 50)

    strategy_performance = {}

    for strategy_name in strategies.keys():
        strategy_results = []

        for model_type, results in all_model_results.items():
            if strategy_name in results['federated']:
                for param_key, param_results in results['federated'][strategy_name].items():
                    if param_results:
                        param_df = pd.DataFrame(param_results)
                        strategy_results.append({
                            'model': model_type,
                            'param': param_key,
                            'accuracy': param_df['accuracy'].mean(),
                            'f1': param_df['f1'].mean()
                        })

        if strategy_results:
            strategy_df = pd.DataFrame(strategy_results)
            avg_acc = strategy_df['accuracy'].mean()
            avg_f1 = strategy_df['f1'].mean()

            strategy_performance[strategy_name] = {
                'avg_accuracy': avg_acc,
                'avg_f1': avg_f1,
                'best_accuracy': strategy_df['accuracy'].max(),
                'best_f1': strategy_df['f1'].max()
            }

            print(f"{strategy_name}:")
            print(f"  Average Accuracy: {avg_acc:.4f}")
            print(f"  Average F1: {avg_f1:.4f}")
            print(f"  Best Accuracy: {strategy_df['accuracy'].max():.4f}")
            print(f"  Best F1: {strategy_df['f1'].max():.4f}")

    summary['strategy_performance'] = strategy_performance

    # Generate insights
    insights = []

    # Best performing model
    model_performances = {}
    for model_type, results in all_model_results.items():
        best_acc = 0
        for approach in ['centralized', 'local', 'federated']:
            if approach in results and results[approach]:
                if approach == 'federated':
                    for strategy_name, strategy_results in results[approach].items():
                        for param_key, param_results in strategy_results.items():
                            if param_results:
                                param_df = pd.DataFrame(param_results)
                                acc = param_df['accuracy'].mean()
                                best_acc = max(best_acc, acc)
                else:
                    approach_df = pd.DataFrame(results[approach])
                    acc = approach_df['accuracy'].mean()
                    best_acc = max(best_acc, acc)

        model_performances[model_type] = best_acc

    best_model = max(model_performances, key=model_performances.get)
    insights.append(f"Best performing model: {best_model.upper()} (Accuracy: {model_performances[best_model]:.4f})")

    # Best strategy
    if strategy_performance:
        best_strategy = max(strategy_performance, key=lambda x: strategy_performance[x]['avg_accuracy'])
        insights.append(f"Best FL strategy: {best_strategy} (Avg Accuracy: {strategy_performance[best_strategy]['avg_accuracy']:.4f})")

    summary['insights'] = insights

    print(f"\n3. KEY INSIGHTS:")
    print("-" * 20)
    for insight in insights:
        print(f"• {insight}")

    return summary

print("Comprehensive summary function ready!")

In [ ]:
# =============================================================================
# MAIN EXECUTION BLOCK
# =============================================================================

def main():
    """
    Main execution function that runs all experiments
    """

    print(f"\n{'='*100}")
    print("FEDERATED LEARNING COMPREHENSIVE COMPARISON STUDY")
    print("Neural Network vs SVM vs Logistic Regression")
    print("WITH BOXPLOTS, MEANS, AND 20-FOLD CROSS-VALIDATION")
    print(f"{'='*100}")

    # Check if data is loaded
    if 'X' not in globals() or 'y' not in globals():
        print("ERROR: Data not loaded. Please run the data loading section first.")
        return

    print(f"Dataset shape: {X.shape}")
    print(f"Class distribution: {np.bincount(y)}")
    print(f"Cross-validation folds: {NUM_FOLDS}")
    print(f"Centralized/Local max iterations: 200")
    print(f"Federated learning max iterations: 20")

    # Store all results
    all_model_results = {}

    # Define models to test with their DEFAULT hyperparameters
    models_to_test = [
        ('nn', 'Neural Network', None),
        ('svm', 'Support Vector Machine', None),
        ('lr', 'Logistic Regression', None)
    ]

    # Run experiments for each model
    for model_type, model_name, custom_hyperparams in models_to_test:
        print(f"\n{'='*80}")
        print(f"STARTING {model_name.upper()} EXPERIMENTS")
        print(f"{'='*80}")

        try:
            # Run complete experiment with custom hyperparameters
            results = run_complete_experiment(
                X, y, model_type, apply_splines=True,
                custom_hyperparams=custom_hyperparams
            )
            all_model_results[model_type] = results

            # Analyze results for this model
            analysis = analyze_results(results, model_type)

            # Create comparison table
            comparison_table = create_comparison_table(results, model_type)

            print(f"\n{model_name.upper()} experiments completed successfully!")

        except Exception as e:
            print(f"ERROR in {model_name} experiments: {e}")
            import traceback
            traceback.print_exc()

    # Generate comprehensive comparison if results available
    if all_model_results:
        print(f"\n{'='*100}")
        print("GENERATING COMPREHENSIVE COMPARISON WITH BOXPLOTS")
        print(f"{'='*100}")

        # Create boxplot visualizations
        try:
            print("Creating performance boxplots...")
            plot_df = create_performance_boxplots(all_model_results)
            print("Creating strategy comparison boxplots...")
            create_strategy_comparison_boxplot(all_model_results)
        except Exception as e:
            print(f"Error creating boxplots: {e}")
            import traceback
            traceback.print_exc()

        # Generate comprehensive summary
        try:
            print("Generating comprehensive summary...")
            comprehensive_summary = generate_comprehensive_summary(all_model_results)
        except Exception as e:
            print(f"Error generating summary: {e}")
            import traceback
            traceback.print_exc()

        # Save results
        try:
            print("Saving results to CSV files...")
            save_results(all_model_results, filename_prefix="fl_results_20folds")
        except Exception as e:
            print(f"Error saving results: {e}")

        print(f"\n{'='*100}")
        print("EXPERIMENT COMPLETED SUCCESSFULLY!")
        print(f"{'='*100}")

        # Display key findings
        if 'comprehensive_summary' in locals() and comprehensive_summary and 'insights' in comprehensive_summary:
            print("\nKEY FINDINGS:")
            for insight in comprehensive_summary['insights']:
                print(f"• {insight}")

        print("\nResults Summary:")
        print(f"Completed {NUM_FOLDS} cross-validation folds for each approach")
        print(f"Tested {len(all_model_results)} different models")
        print(f"Evaluated multiple federated learning strategies with model-specific hyperparameters")
        print(f"Used 200 iterations for centralized and local training")
        print(f"Used 20 iterations for federated learning")
        print(f"Generated boxplot visualizations with means")
        print(f"Saved detailed results to CSV files")

        return all_model_results, comprehensive_summary if 'comprehensive_summary' in locals() else None

    else:
        print("No results to analyze. Please check the error messages above.")
        return None, None

# Save results function
def save_results(all_model_results, filename_prefix="fl_results"):
    """Save results to CSV files"""

    print("Saving results to CSV files...")

    # Create a metadata file
    metadata = {
        'experiment_info': {
            'num_folds': NUM_FOLDS,
            'num_partitions': NUM_PARTITIONS,
            'fl_rounds': NUM_ROUNDS,
            'centralized_local_iterations': 200,
            'federated_iterations': 20,
            'timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
        }
    }

    # Save metadata
    with open(f"{filename_prefix}_metadata.txt", 'w') as f:
        for key, value in metadata['experiment_info'].items():
            f.write(f"{key}: {value}\n")

    # Save detailed results for each model
    for model_type, results in all_model_results.items():
        # Save centralized results
        if results['centralized']:
            centralized_df = pd.DataFrame(results['centralized'])
            centralized_df['fold'] = range(1, len(centralized_df) + 1)
            centralized_df.to_csv(f"{filename_prefix}_{model_type}_centralized.csv", index=False)
            print(f"  Saved: {filename_prefix}_{model_type}_centralized.csv ({len(centralized_df)} folds)")

        # Save local results
        if results['local']:
            local_df = pd.DataFrame(results['local'])
            local_df['fold'] = range(1, len(local_df) + 1)
            local_df.to_csv(f"{filename_prefix}_{model_type}_local.csv", index=False)
            print(f"  Saved: {filename_prefix}_{model_type}_local.csv ({len(local_df)} folds)")

        # Save federated results
        for strategy_name, strategy_results in results['federated'].items():
            for param_key, param_results in strategy_results.items():
                if param_results:
                    param_df = pd.DataFrame(param_results)
                    param_df['fold'] = range(1, len(param_df) + 1)
                    filename = f"{filename_prefix}_{model_type}_{strategy_name}_{param_key}.csv"
                    param_df.to_csv(filename, index=False)
                    print(f"  Saved: {filename} ({len(param_df)} folds)")

    # Create summary statistics file
    summary_data = []
    for model_type, results in all_model_results.items():
        # Centralized summary
        if results['centralized']:
            centralized_df = pd.DataFrame(results['centralized'])
            summary_data.append({
                'Model': model_type.upper(),
                'Approach': 'Centralized',
                'Strategy': 'N/A',
                'Parameter': 'N/A',
                'Accuracy_Mean': centralized_df['accuracy'].mean(),
                'Accuracy_Std': centralized_df['accuracy'].std(),
                'F1_Mean': centralized_df['f1'].mean(),
                'F1_Std': centralized_df['f1'].std(),
                'Precision_Mean': centralized_df['precision'].mean(),
                'Precision_Std': centralized_df['precision'].std(),
                'Recall_Mean': centralized_df['recall'].mean(),
                'Recall_Std': centralized_df['recall'].std(),
                'Num_Folds': len(centralized_df)
            })

        # Local summary
        if results['local']:
            local_df = pd.DataFrame(results['local'])
            summary_data.append({
                'Model': model_type.upper(),
                'Approach': 'Local',
                'Strategy': 'N/A',
                'Parameter': 'N/A',
                'Accuracy_Mean': local_df['accuracy'].mean(),
                'Accuracy_Std': local_df['accuracy'].std(),
                'F1_Mean': local_df['f1'].mean(),
                'F1_Std': local_df['f1'].std(),
                'Precision_Mean': local_df['precision'].mean(),
                'Precision_Std': local_df['precision'].std(),
                'Recall_Mean': local_df['recall'].mean(),
                'Recall_Std': local_df['recall'].std(),
                'Num_Folds': len(local_df)
            })

        # Federated summary
        for strategy_name, strategy_results in results['federated'].items():
            for param_key, param_results in strategy_results.items():
                if param_results:
                    param_df = pd.DataFrame(param_results)
                    summary_data.append({
                        'Model': model_type.upper(),
                        'Approach': 'Federated',
                        'Strategy': strategy_name,
                        'Parameter': param_key,
                        'Accuracy_Mean': param_df['accuracy'].mean(),
                        'Accuracy_Std': param_df['accuracy'].std(),
                        'F1_Mean': param_df['f1'].mean(),
                        'F1_Std': param_df['f1'].std(),
                        'Precision_Mean': param_df['precision'].mean(),
                        'Precision_Std': param_df['precision'].std(),
                        'Recall_Mean': param_df['recall'].mean(),
                        'Recall_Std': param_df['recall'].std(),
                        'Num_Folds': len(param_df)
                    })

    # Save summary statistics
    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv(f"{filename_prefix}_summary_statistics.csv", index=False)
    print(f"  Saved: {filename_prefix}_summary_statistics.csv")

    print("Results saved successfully!")
    return summary_df

print("Main execution functions ready!")

## Run the Complete Experiment

Execute the following cell to run all experiments. This will:

1. **Sequential Execution**: Run NN, SVM, and LR experiments with cross-validation
2. **Comprehensive Analysis**: Compare all approaches and strategies
3. **Visualization**: Generate performance plots and comparisons
4. **Summary Report**: Provide insights and recommendations

**Note**: This may take several hours to complete depending on your system.

In [ ]:
# =============================================================================
# EXECUTE THE COMPLETE EXPERIMENT
# =============================================================================

# Run all experiments
if __name__ == "__main__":
    # Execute main experiment
    all_results, summary = main()

    # Enhanced result saving if successful
    if all_results is not None:
        try:
            summary_stats_df = save_results(all_results)

            print(f"\n{'='*100}")
            print("FINAL EXPERIMENTAL SUMMARY")
            print(f"{'='*100}")
            print(f"Completed comprehensive federated learning comparison")
            print(f"Generated boxplot visualizations with means and distributions")
            print(f"Performed {NUM_FOLDS} cross-validation folds per approach")
            print(f"Saved detailed results and summary statistics to CSV files")
            print(f"Model-specific hyperparameter optimization completed")
            print(f"{'='*100}")

            # Display top performing configurations
            if 'summary_stats_df' in locals() and not summary_stats_df.empty:
                print("\nTOP PERFORMING CONFIGURATIONS:")
                top_configs = summary_stats_df.nlargest(5, 'Accuracy_Mean')[
                    ['Model', 'Approach', 'Strategy', 'Parameter', 'Accuracy_Mean', 'Accuracy_Std']
                ]
                print(top_configs.to_string(index=False))

        except Exception as e:
            print(f"Error in enhanced result saving: {e}")
            # Fallback to basic saving
            save_results(all_results, filename_prefix="fl_results_20folds_basic")

        print("\nExperiment completed successfully!")
        print("Check the generated CSV files for detailed results")
        print("Review the boxplot visualizations for performance distributions")

    else:
        print("Experiment failed. Please check the error messages above.")
        print("Common issues: data not loaded, CUDA problems, or configuration errors")